<a href="https://colab.research.google.com/github/adidam/rag-impl/blob/main/Real_World_RAG_Impl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install sentence-transformers
!pip install rank-bm25
!pip install torch transformers
!pip install huggingface_hub
!pip install langchain-community
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ...

## **Adding the Imports**

In [1]:
import numpy as np
import os
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import random_split
import torch.optim as optim
import matplotlib.pyplot as plt
import nltk

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## **Loading the RAGBench Dataset**

## **Chunking the Dataset**

In [28]:
# New code - 12/4 10 pm

from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("BAAI/LLM-Embedder")
#tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Sliding window configuration
TOKEN_LIMIT = 512
SLIDING_WINDOW_OVERLAP = 100  # Overlap between consecutive chunks (in tokens)

# Function for chunking with token limit and sliding window
def chunk_with_token_limit(text, token_limit, overlap):
    sentences = sent_tokenize(text)  # Split text into sentences
    chunks = []  # Store resulting chunks
    current_chunk = []  # Temporarily hold sentences for the current chunk
    current_chunk_tokens = 0  # Token count for the current chunk

    for sentence in sentences:
        # Tokenize the sentence and calculate its token count
        sentence_tokens = tokenizer.tokenize(sentence)
        num_tokens = len(sentence_tokens)

        # print(f"Tokens: {sentence_tokens[0]}")

        # If adding this sentence exceeds the token limit
        if current_chunk_tokens + num_tokens > token_limit:
            # Save the current chunk
            chunk_text = " ".join(current_chunk)
            chunks.append(chunk_text)

            # Prepare the next chunk with overlap
            overlap_tokens = tokenizer.tokenize(" ".join(current_chunk[-1:]))
            current_chunk = [sentence for sentence in current_chunk[-(overlap // len(overlap_tokens)) :]] if current_chunk else []
            current_chunk_tokens = sum(len(tokenizer.tokenize(sent)) for sent in current_chunk)

        # Add the sentence to the current chunk
        current_chunk.append(sentence)
        current_chunk_tokens += num_tokens

    # Add the last chunk if it exists
    if current_chunk:
        chunk_text = " ".join(current_chunk)
        chunks.append(chunk_text)

    return chunks

def process_document_with_identifiers(document):
    processed_data = []
    title_count = -1  # to start from 0
    print("document>>>>>>>",document)
    for section in document:
        section_chunks = []
        passage_count = [ord('a')]  # Passage identifier as a list to handle nested increments
        title_count += 1  # Increment title count

        # Tokenize the section into sentences
        sentences = sent_tokenize(section)
        for sentence in sentences:
            if sentence.startswith("Title:"):
                # New document detected
                identifier = f"{title_count}{''.join(chr(c) for c in passage_count)}"  # Identifier for the title
                chunked_texts = chunk_with_token_limit(sentence, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)
                for chunk in chunked_texts:
                    section_chunks.append([identifier, chunk])
                passage_count = [ord('a')]  # Reset passage count for the new document
            else:
                # Sentence under the current document
                identifier = f"{title_count}{''.join(chr(c) for c in passage_count)}"
                chunked_texts = chunk_with_token_limit(sentence, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)
                #print("chunked_texts>>>>process_document_with_identifiers>>>>> "+ "".join(chunked_texts))
                for chunk in chunked_texts:
                    section_chunks.append([identifier, chunk])

                # Increment passage_count intelligently
                i = len(passage_count) - 1
                while i >= 0:
                    passage_count[i] += 1
                    if passage_count[i] > ord('z'):
                        passage_count[i] = ord('a')
                        if i == 0:
                            passage_count.insert(0, ord('a'))  # Add a new character to the identifier
                        i -= 1
                    else:
                        break


        print("section_chunks>>>>>>>",section_chunks)
        processed_data.append(section_chunks)

    return processed_data

# **Load and view first 5 rows of the Dataset**

In [ ]:
# Print the top 5 rows of the dataset, for debugging purpose, we only generate embeddings for these 5 rows
from datasets import load_dataset
datasets = ['techqa']
data = load_dataset("rungalileo/ragbench", datasets[0], split="train")
top_5_rows = data.select(range(2))

for i, row in enumerate(top_5_rows):
    print(f"Row {i + 1}:")
    for field, value in row.items():
        print(f"  {field}: {value}")
    print()

## **Generate Embeddings**

In [29]:
from datasets import load_dataset
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

#datasets = ['pubmedqa','tatqa', 'techqa'] # d4

datasets = ['techqa']

# Initialize storage for documents, IDs, and metadata
all_documents = []
all_ids = []
all_metadatas = []

# Process each dataset
doc_idx = 0  # Global document index for unique IDs
for dataset in datasets:
    data = load_dataset("rungalileo/ragbench", dataset, split="train")
    #only select first 5 records for debugging duplicate records. **PLEASE REMOVE THIS AFTER DEBUGGING**
    data = data.select(range(2))
    for idx, row in tqdm(enumerate(data), desc=f"Processing {dataset}"):
        # Extract document text
        doc_text = row.get('documents', '')

        # Skip if no documents found
        if not doc_text:
            continue

        # Process the document
        processed_output = process_document_with_identifiers(doc_text)
        added_item_idxs = set()

        # Populate the lists
        for section_idx, section in enumerate(processed_output):
            for item_idx, (prefix, content) in enumerate(section):
                # Skip if this item_idx has already been processed
                if item_idx in added_item_idxs:
                    continue

                # Add the item_idx to the set to track it
                added_item_idxs.add(item_idx)

                # Add the document
                document = f"[{prefix}] {content}"
                all_documents.append(document)

                # Construct a globally unique ID
                doc_id = f"{dataset}_{doc_idx}_{section_idx}_{item_idx}"
                all_ids.append(doc_id)

                # Construct metadata
                metadata = {
                    "dataset": dataset,
                    "global_index": doc_idx,
                    "section_index": section_idx,
                    "item_index": item_idx,
                    "prefix": prefix,
                    "type": "Title" if prefix.endswith("a") else "Passage",
                }
                all_metadatas.append(metadata)

        doc_idx += 1  # Increment global document index

# Step 4: Generate Embeddings
#embedder = SentenceTransformer("all-MiniLM-L6-v2")  # Pretrained sentence transformer
embedder = SentenceTransformer("BAAI/LLM-Embedder")  # Pretrained sentence transformer
batch_size = 2500  # Adjust based on available memory

# Generate embeddings in batches
all_embeddings = []
for i in tqdm(range(0, len(all_documents), batch_size), desc="Generating embeddings"):
    batch_docs = all_documents[i:i + batch_size]
    batch_embeddings = embedder.encode(batch_docs, show_progress_bar=True)
    all_embeddings.extend(batch_embeddings)

Processing techqa: 3it [00:00, 28.18it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (839 > 512). Running this sequence through the model will result in indexing errors
Processing techqa: 1192it [00:34, 34.90it/s]
Generating embeddings:   0%|          | 0/48 [00:00<?, ?it/s]

Batches:   0%|          | 0/79 [00:00<?, ?it/s]

Generating embeddings:   2%|▏         | 1/48 [00:13<10:34, 13.51s/it]

Batches:   0%|          | 0/79 [00:00<?, ?it/s]

Generating embeddings:   4%|▍         | 2/48 [00:25<09:30, 12.40s/it]

...

Generating embeddings:  96%|█████████▌| 46/48 [09:59<00:26, 13.22s/it]

Batches:   0%|          | 0/79 [00:00<?, ?it/s]

Generating embeddings:  98%|█████████▊| 47/48 [10:11<00:12, 12.98s/it]

Batches:   0%|          | 0/25 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 48/48 [10:15<00:00, 12.83s/it]


## **Store Embeddings into Milvus**

In [6]:
!pip install pymilvus pymilvus[model]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.68.1
    Uninstalling grpcio-1.68.1:
      Successfully uninstalled grpcio-1.68.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.2 which is incompatible.


# **Functions to check uniqueness of data being inserted to db**

In [ ]:
import hashlib

# Function to generate a hash based on content and key metadata
def generate_hash(content, metadata):
    """Generate a unique hash for the document content and key metadata."""
    key_fields = f"{content}|{metadata.get('item_index')}|{metadata.get('prefix')}"
    return hashlib.md5(key_fields.encode('utf-8')).hexdigest()

# Function to retrieve existing hashes from the database
def get_existing_hashes(collection):
    """Retrieve all existing hashes (IDs) currently in the database."""
    all_records = collection.get(include=["documents", "metadatas"])  # Fetch documents and metadata
    existing_hashes = set()
    for doc, metadata in zip(all_records["documents"], all_records["metadatas"]):
        doc_hash = generate_hash(doc, metadata)
        existing_hashes.add(doc_hash)
    return existing_hashes

In [32]:
import numpy as np
from pymilvus import connections
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection
from pymilvus import MilvusClient
from pymilvus import utility

class VectorDataStore:
    db_url = "http://localhost:19530"
    #description = f"collection created for {self.name}"

    def __init__(self, path="/content/ragbench.db"):
        self.client = MilvusClient(path)



    def create_collection(self, name, vec_dim=128):
        if self.client.has_collection(name):
            self.default_collection_name = name

        self.description = f"collection to store {name}"

        index_params = self.client.prepare_index_params()
        index_params.add_index(
            field_name="embedding",
            index_type="AUTOINDEX",
            metric_type="COSINE"
        )
        schema = self.client.create_schema(
            auto_id=False,
            enable_dynamic_fields=True,
        )
        schema.add_field(field_name="pk", datatype=DataType.VARCHAR, max_length=64, is_primary=True)
        schema.add_field(field_name="metadata", datatype=DataType.JSON)
        schema.add_field(field_name="documents", datatype=DataType.VARCHAR, max_length=512)
        schema.add_field(field_name="embedding", datatype=DataType.FLOAT_VECTOR, dim=vec_dim)

        collection = self.client.create_collection(collection_name=name,
                                       schema=schema,
                                       index_params=index_params)
        self.current_collection = collection
        return collection


    def get_collection(self, name):
        if not self.client.has_collection(name):
            raise ValueError(f"Collection '{name}' does not exist.")
        self.current_collection = Collection(name)
        return self.current_collection
    
    def get_all_records(self, collection):
        self.client.query(
            collection_name=collection,
            filter="pk >= 0",
            output_fields=["documents", "metadata"],
        )

    def insert(self, collection_name: str, metadata: list[dict[str, any]],
                documents: list[str], embeddings: np.ndarray, ids: list[int]):

        if not self.client.has_collection(collection_name):
            raise ValueError(f"Collection '{collection_name}' does not exist. Create it first.")

        if len(metadata) != len(embeddings) != len(documents) != len(ids):
           raise ValueError("Metadata, documnets, ids and embeddings must have the same length.")

        data = []
        for meta, doc, emb, id in zip(metadata, documents, embeddings, ids):
          datum = {
              "pk": id,
              "metadata": meta,
              "documents": doc,
              "embedding": emb.tolist(),
          }
          data.append(datum)

        self.client.insert(collection_name, data)
        print(f"Inserted {len(metadata)} records into collection '{collection_name}'.")

    def drop_collection(self, collection_name: str):
        if not self.client.has_collection(collection_name):
            raise ValueError(f"Collection '{collection_name}' does not exist.")
        self.client.drop_collection(collection_name)
        print(f"Dropped collection '{collection_name}'.")

    def delete_all(self, collection_name: str):
        if not self.client.has_collection(collection_name):
            raise ValueError(f"Collection '{collection_name}' does not exist.")
        self.client.delete(collection_name, expr="pk >= 0")
        self.client.flush([collection_name])

    def search(self, query_embedding: np.ndarray, top_k: int = 10) -> list[dict[str, any]]:
        """
        Search across all collections for the top-k closest embeddings.
        :param query_embedding: The embedding vector to search for.
        :param top_k: Number of top results to retrieve.
        :return: A list of dictionaries containing collection name, id, metadata, and distance.
        """
        results = []
        #collections = self.client.list_collections()
        collections = ["ragbench_collection_techqa_v09"]
        start_time = time.time()
        for collection_name in collections:
            if not self.client.has_collection(collection_name):
                continue

            # Set params to COSINE to match chromadb
            search_params = {"metric_type": "COSINE", "params": {"ef": 128}}

            search_results = self.client.search(
                collection_name=collection_name,
                data=[query_embedding],
                anns_field="embedding",
                search_params=search_params,
                limit=top_k,
                output_fields=["metadata", "documents"]
            )

            for hits in search_results:
                for hit in hits:
                    print(f"Collection: {collection_name}, data: {str(hit)}")
                    results.append({
                        "collection": collection_name,
                        "id": hit["id"],
                        "metadata": hit["entity"]["metadata"],
                        "distance": hit["distance"],
                        "documents": hit["entity"]["documents"]
                      })

        results = sorted(results, key=lambda x: x["distance"])[:top_k]
        end_time = time.time()
        print(f"Search completed. Found {len(results)} results. in {end_time - start_time} secs")
        return results

    def extract_documents(self, search_results: list[dict[str, any]]) -> list[np.ndarray]:
      """
      Extract embedding values from search results.
      :param search_results: List of dictionaries containing search results.
      :return: List of embedding vectors as NumPy arrays.
      """
      return [np.array(result["documents"]) for result in search_results if "documents" in result]

## **Instantiate Milvus and add data to milvus db**

In [35]:
collection_name = "ragbench_collection_techqa_v01"
datastor = VectorDataStore()

insert_data = False
store_client = "Milvus"

if datastor.client.has_collection(collection_name):
  num_records = datastor.has_entities(collection_name)
  if num_records == 0:
    insert_data = True
else:
  datastor.create_collection(collection_name, embedder.get_sentence_embedding_dimension())
  insert_data = True



# **Store Embeddings into a chroma DB and Milvus**

In [ ]:
!pip install chromadb

In [ ]:
import chromadb

client = chromadb.PersistentClient(path="./content/rag_chroma_db_d4")

# chromba_db_collection_name = "ragbench_chroma_db_collection_techqa_v21"
chroma_db_collection = client.create_collection(name=collection_name)

count = chroma_db_collection.count()
store_client = "Chromadb"

if count > 0:
    insert_data = True
else:
    insert_data = False


## **Insert data into data base**
### the store_client value stores if the code should call milvus or chromadb code
#### if insert_data is set to true it means there is no data in the collection


In [ ]:
# Adding data to ChromaDB with enhanced duplicate check
existing_hashes = get_existing_hashes(chroma_db_collection)

for i in tqdm(range(0, len(all_documents), batch_size), desc="Adding data to DB"):
    batch_embeddings = all_embeddings[i:i + batch_size]
    batch_metadatas = all_metadatas[i:i + batch_size]
    batch_documents = all_documents[i:i + batch_size]
    batch_ids = []

    # Generate hashes for each document in the batch
    for doc, metadata in zip(batch_documents, batch_metadatas):
        doc_hash = generate_hash(doc, metadata)
        if doc_hash not in existing_hashes:
            batch_ids.append(doc_hash)
            existing_hashes.add(doc_hash)  # Add hash to local set to avoid duplicates in the same batch
        else:
            print(f"Skipping duplicate document: {doc[:50]}...")  # Print a preview of the duplicate doc

    # Add non-duplicate documents to the database
    if batch_ids:  # Ensure there are non-duplicate documents to add
        if store_client == "Chromadb" and insert_data:
            chroma_db_collection.add(
                embeddings=batch_embeddings[:len(batch_ids)],  # Trim embeddings to match batch_ids
                metadatas=batch_metadatas[:len(batch_ids)],    # Trim metadatas to match batch_ids
                documents=batch_documents[:len(batch_ids)],    # Trim documents to match batch_ids
                ids=batch_ids
            )
        

# **Insert data to milvus**

In [ ]:
# Adding data to ChromaDB with enhanced duplicate check
existing_hashes = datastor.get_all_records(collection_name)

for i in tqdm(range(0, len(all_documents), batch_size), desc="Adding data to DB"):
    batch_embeddings = all_embeddings[i:i + batch_size]
    batch_metadatas = all_metadatas[i:i + batch_size]
    batch_documents = all_documents[i:i + batch_size]
    batch_ids = []

    # Generate hashes for each document in the batch
    for doc, metadata in zip(batch_documents, batch_metadatas):
        doc_hash = generate_hash(doc, metadata)
        if doc_hash not in existing_hashes:
            batch_ids.append(doc_hash)
            existing_hashes.add(doc_hash)  # Add hash to local set to avoid duplicates in the same batch
        else:
            print(f"Skipping duplicate document: {doc[:50]}...")  # Print a preview of the duplicate doc

    # Add non-duplicate documents to the database
    if batch_ids:  # Ensure there are non-duplicate documents to add
        # Add the batch to the Milvus collection
        if store_client == "Milvus" and insert_data:
            datastor.insert(collection_name,
                metadata=batch_metadatas,
                documents=batch_documents,
                embeddings=np.array(batch_embeddings),
                ids=batch_ids
            )

## **Verifying retrival logic for the Relevant documents**

In [37]:
# Retrieve from Chroma DB
import time

question ="Why does the other instance of my multi-instance qmgr seem to hang after a failover? Queue manager will not start after failover"
query_embedding = embedder.encode(question).tolist()

# Search for relevant chunks in the vector database
# Retrieve from ChromaDB
start_time = time.time()
results_chroma_db = chroma_db_collection.query(query_embeddings=[query_embedding], n_results=5)
end_time = time.time()
print(f"ChromaDB - Search completed. Found {len(results_chroma_db)} results. in {end_time - start_time} secs")

for idx, (doc, doc_id, metadata) in enumerate(zip(results_chroma_db["documents"][0], results_chroma_db["ids"][0], results_chroma_db["metadatas"][0])):
    print(f"Result {idx + 1}:")
    print(f"  Document: {doc}")
    print(f"  ID: {doc_id}")
    print(f"  Metadata: {metadata}")
    print()


Collection: ragbench_collection_techqa_v01, data: {'id': 'techqa_9_2_16', 'distance': 0.9003783464431763, 'entity': {'documents': '[0q] ENVIRONMENT\nCICS TS V4.1 or later; CICSPlex SM V4.1 or later WUI Server using CMCI;\n\n\n\nDIAGNOSING THE PROBLEM\nThe majority of IZE0105E or CNX0104E connection failure messages can be resolved by examining the CICS TS MSGUSR logs for CICS errors related to attaching the CWXN or CWWU transaction identifiers.', 'metadata': {'dataset': 'techqa', 'global_index': 9, 'section_index': 2, 'item_index': 16, 'prefix': '0q', 'type': 'Passage'}}}
Collection: ragbench_collection_techqa_v01, data: {'id': 'techqa_605_2_16', 'distance': 0.9003783464431763, 'entity': {'documents': '[0q] ENVIRONMENT\nCICS TS V4.1 or later; CICSPlex SM V4.1 or later WUI Server using CMCI;\n\n\n\nDIAGNOSING THE PROBLEM\nThe majority of IZE0105E or CNX0104E connection failure messages can be resolved by examining the CICS TS MSGUSR logs for CICS errors related to attaching the CWXN or 

In [ ]:
import time

question ="Why does the other instance of my multi-instance qmgr seem to hang after a failover? Queue manager will not start after failover"
query_embedding = embedder.encode(question).tolist()

# Retrieve from Milvus
start_time = time.time()
results_milvus = datastor.search(query_embedding, top_k=5)
end_time = time.time()
print(f"Search completed. Found {len(results_milvus)} results. in {end_time - start_time} secs")

for doc in results_milvus:
    print("Relevant Docs from Milvus:\n", doc['documents'])

## **Retrival of Relevant Chunks**

In [10]:
# Function to retrieve relevant chunks
def retrieve_docs_milvus(query, encoder, top_k=5):
    # Generate embedding for the query
    query_embedding = encoder.encode(query)
    # Perform vector search to find relevant chunks
    results = datastor.extract_documents(datastor.search(query_embedding, top_k))
    print(f"results: {results}")
    # Extract the retrieved chunks
    chunks = results
    # should sort and push context - but later
    return chunks

In [ ]:
# Function to retrieve relevant chunks
def retrieve_docs(query, top_k=5):
    # Generate embedding for the query
    query_embedding = embedder.encode(query).tolist()
    # Perform vector search to find relevant chunks
    results = chroma_db_collection.query(query_embeddings=[query_embedding], n_results=top_k)
    # Extract the retrieved chunks
    chunks = results["documents"]
    # should sort and push context - but later
    return chunks

In [ ]:
def retrieve_docs_query(query, top_k=5):
    query_embedding = embedder.encode(query)

    if store_client == 'Milvus':
        results = datastor.search(query_embedding, top_k)
        results = datastor.extract_documents(results)
    elif store_client == "Chromadb":
        results = chroma_db_collection.query(query_embeddings=[query_embedding.tolist()], n_results=top_k)
        results = results['documents']
    return results

## **Query Classification**

In [11]:
!pip install transformers datasets torch

In [ ]:
# Get the databricks dolly dataset (as mentioned in  "Searching for Best Practices in Retrieval-Augmented Generation" paper) for training the query classifer.
from datasets import load_dataset

ds = load_dataset("databricks/databricks-dolly-15k")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

# Step 1: Load Pre-trained BERT and Tokenizer
model_name = "bert-base-multilingual-cased"
#model_name = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Step 2: Prepare the Dataset
# Replace with your labeled data
# Example data: {"query": ["What is COVID19?", "What do the S1 and S2 subunits contain?"], "label": [0, 1]}

data = {
    "query": [
        "What is SQLCODE=-1585",
        "The configuration task database-transfer failed with DB2 SQL Error: SQLCODE=-1585, SQLSTATE=54048 While attempting to run the database-transfer task the following error is logged to the ConfigTrace.log: action-process-constraints: Fri Oct 10 13:20:34 CDT 2014 Target started: action-process-constraints [java] Executing java with empty input string [java] [10/10/14 13:20:35.877 CDT] Attempting to create a new Instance of com.ibm.db2.jcc.DB2Driver [java] [10/10/14 13:20:36.016 CDT] Instance of com.ibm.db2.jcc.DB2Driver created successfully [java] [10/10/14 13:20:36.016 CDT] Attempting to make connection using: jdbc:db2://:60500/:returnAlias=0; :: d2svc :: PASSWORD_REMOVED [java] [10/10/14 13:20:36.954 CDT] Connection successfully made [java] [10/10/14 13:20:37.073 CDT] ERROR: Error occurred gathering data from the source database [java] com.ibm.db2.jcc.am.SqlException: DB2 SQL Error: SQLCODE=-1585, SQLSTATE=54048, SQLERRMC=null, DRIVER=4.18.60 [java] at com.ibm.db2.jcc.am.kd.a(kd.java:752)",
        "What is Websphere Application Server",
        "How do I change from shared to unshared connection? in WAS, how do I change from shared to unshared connection. I am seeing connections max out and take a long time to release.",
        "What is JMS",
        "Why is my MQ Java / JMS application getting 2035 NOT_AUTHORIZED error after upgrade of MQ? Why is my MQ Java / JMS application getting 2035 NOT_AUTHORIZED error after upgrade of MQ?",
        "What is TLS",
        "TLS protocol with ITCAM for Datapower We have a DataPower appliance with TLS security protocol enabled. Can we configure ITCAM for DataPower appliance v7.1 to specifically use the TLS protocol v1.2 (not v1.0)?"

    ],
    "label": [0, 1, 0, 1, 0, 1, 0, 1]  # 0: retrieval not needed, 1: retrieval needed
}

# Convert to Hugging Face Dataset
dataset = Dataset.from_dict(data)

#dataset = ds
# Tokenize the Dataset
def preprocess(data):
    return tokenizer(data["query"], padding="max_length", truncation=True, max_length=128)

encoded_dataset = dataset.map(preprocess, batched=True)
encoded_dataset = encoded_dataset.train_test_split(test_size=0.2)

# Step 3: Define the Training Loop
training_args = TrainingArguments(
    output_dir="./query_classifier_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

# Custom Metric for Accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = (predictions == torch.tensor(labels)).float().mean().item()
    return {"accuracy": accuracy}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Step 4: Train the Model
trainer.train()

# Step 5: Save the Model
model.save_pretrained("./query_classifier")
tokenizer.save_pretrained("./query_classifier")

# Step 6: Inference Function
def classify_query(query, model_path="./query_classifier"):
    # Load saved model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)

    # Tokenize the input query
    inputs = tokenizer(query, return_tensors="pt", padding="max_length", truncation=True, max_length=128)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()

    return predicted_class


# **LLM Inference with groq**

In [12]:
! pip install groq
! pip install -q langchain langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 6.1 MB/s eta 0:00:00


In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

# Initializing the context variable which later gets populated with retrieved chunks
context = ""

In [21]:
def query_response_from_llm(query: str):

    # retrieve chunks from milvus db
    chunks = retrieve_docs_milvus(query)
    #print("chunks >>>query_response_from_llm>> ", chunks)

    # retrieve chunks from chroma db
    #chunks = retrieve_docs(query)

    # Flatten the list if necessary
    if any(isinstance(chunk, list) for chunk in chunks):
      chunks = [item for sublist in chunks for item in (sublist if isinstance(sublist, list) else [sublist])]

    chat = ChatGroq(temperature=0.3, groq_api_key="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", model_name="llama3-8b-8192")

    prompt=ChatPromptTemplate.from_template(
      """
      Please provide a response to the query below, strictly adhering to the
      information presented in the following documents.
      Do not generate any text beyond what is explicitly stated in the documents.

      Context: {context}

      Question: {query}

      Answer:
      """
    )

    chain = prompt | chat

    context = "".join(chunks)

    print("context>>>from 1st RAG>>>>>> ",context)

    groq_response = chain.invoke({"context": context, "query": query})

    print("groq_response>>>from 1st RAG>>>>>> ",groq_response)

    answer = groq_response
    return answer, context

In [ ]:
def query_response_from_llm_no_rag(query: str):

    chat = ChatGroq(temperature=0.3, groq_api_key="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", model_name="llama3-8b-8192")

    prompt=ChatPromptTemplate.from_template(
      """
      Please provide a response to the query below

      Question: {query}

      Answer:
      """
    )

    chain = prompt | chat

    groq_response_no_rag = chain.invoke({"query": query})

    print("groq_response_no_rag>>>no RAG>>>>>> ",groq_response_no_rag)

    answer = groq_response_no_rag
    return answer

## **USER QUERY**

In [22]:
#query = "What are the most frequent clinical manifestations of human adenovirus type 55 (HAdV-55) induced ARDS?"
#query ="The configuration task database-transfer failed with DB2 SQL Error: SQLCODE=-1585, SQLSTATE=54048 While attempting to run the database-transfer task the following error is logged to the ConfigTrace.log: action-process-constraints: Fri Oct 10 13:20:34 CDT 2014 Target started: action-process-constraints [java] Executing java with empty input string [java] [10/10/14 13:20:35.877 CDT] Attempting to create a new Instance of com.ibm.db2.jcc.DB2Driver [java] [10/10/14 13:20:36.016 CDT] Instance of com.ibm.db2.jcc.DB2Driver created successfully [java] [10/10/14 13:20:36.016 CDT] Attempting to make connection using: jdbc:db2://:60500/:returnAlias=0; :: d2svc :: PASSWORD_REMOVED [java] [10/10/14 13:20:36.954 CDT] Connection successfully made [java] [10/10/14 13:20:37.073 CDT] ERROR: Error occurred gathering data from the source database [java] com.ibm.db2.jcc.am.SqlException: DB2 SQL Error: SQLCODE=-1585, SQLSTATE=54048, SQLERRMC=null, DRIVER=4.18.60 [java] at com.ibm.db2.jcc.am.kd.a(kd.java:752)"
query = "Why does the other instance of my multi-instance qmgr seem to hang after a failover? Queue manager will not start after failover"

#query = "What is SQLCODE=-1585"

# Query Classification
#classification = classify_query(query)

classification = 1

#print('classification >>>> ',classification)

if classification == 1:
    print("Perform retrieval using RAG.")
    # Call RAG pipeline for retrieval and response generation
    answer,context = query_response_from_llm(query)
else:
    print("No RAG - only use LLM for response generation.")
    # Call LLM directly
    answer = query_response_from_llm_no_rag(query)

print("context  >>from llm>>> ", context)
print("answer  >>from llm>>> ", answer)

classification >>>>  1
Perform retrieval using RAG.
Collection: ragbench_collection_techqa_v01, data: {'id': 'techqa_352_0_1', 'distance': 0.9678059816360474, 'entity': {'documents': '[-1a] database-transfer; DB2; SQLCODE=-1585; SQLSTATE=54048 TECHNOTE (TROUBLESHOOTING)\n\nPROBLEM\nWhile attempting to run the database-transfer task the customer gets the following error:\n\naction-process-constraints:\nFri Oct 10 13:20:34 CDT 2014\nTarget started: action-process-constraints\n[java] Executing java with empty input string\n[java] [10/10/14 13:20:35.877 CDT] Attempting to create a new\nInstance of com.ibm.db2.jcc.DB2Driver\n[java] [10/10/14 13:20:36.016 CDT] Instance of\ncom.ibm.db2.jcc.DB2Driver created successfully\n[java] [10/10/14 13:20:36.016 CDT] Attempting to make connection\nusing: jdbc:db2://<hostname>:60500/<database_name>:returnAlias=0; ::\nd2svc :: PASSWORD_REMOVED\n[java] [10/10/14 13:20:36.954 CDT] Connection successfully made\n[java] [10/10/14 13:20:37.073 CDT] ERROR: Error 

## **PROMPT for generating metrics as JSON response**

In [23]:
def generate_prompt():
    """
    Generate a prompt template for assessing the support and relevance of an LLM-generated response.
    """
    return """
    I asked someone to answer a question based on one or more documents.
    Your task is to review their response and assess whether or not each sentence
    in that response is supported by text in the documents. And if so, which
    sentences in the documents provide that support. You will also tell me which
    of the documents contain useful information for answering the question, and
    which of the documents the answer was sourced from.
    Here are the documents, each of which is split into sentences.Alongside each
    sentence is associated key, such as ’[0a].’ or ’[0b].’ that you can use to refer
    to it:

    ‘‘‘
    {documents}
    ‘‘‘
    The question was:
    ‘‘‘
    {question}
    ‘‘‘

    Here is their response, split into sentences. Alongside each sentence is
    associated key, such as ’a.’ or ’b.’ that you can use to refer to it. Note
    that these keys are unique to the response, and are not related to the keys
    in the documents:
    ‘‘‘
    {answer}
    ‘‘‘
    You must respond with a JSON object matching this schema:
    ‘‘‘
    {{
    "relevance_explanation": string,
    "all_relevant_sentence_keys": [string],
    "overall_supported_explanation": string,
    "overall_supported": boolean,
    "sentence_support_information": [
    {{
    "response_sentence_key": string,
    "explanation": string,
    "supporting_sentence_keys": [string],
    "fully_supported": boolean
    }},
    ],
    "all_utilized_sentence_keys": [string]
    }}
    ‘‘‘
    The relevance_explanation field is a string explaining which documents
    contain useful information for answering the question. Provide a step-by-step
    breakdown of information provided in the documents and how it is useful for
    answering the question.
    The all_relevant_sentence_keys field is a list of all document sentences keys
    (e.g. ’0a’) that are relevant to the question. Include every sentence that is
    useful and relevant to the question, even if it was not used in the response,
    or if only parts of the sentence are useful. Ignore the provided response when
    making this judgement and base your judgement solely on the provided documents
    and question. Omit sentences that, if removed from the document, would not
    impact someone’s ability to answer the question.
    The overall_supported_explanation field is a string explaining why the response
    *as a whole* is or is not supported by the documents. In this field, provide a
    step-by-step breakdown of the claims made in the response and the support (or
    lack thereof) for those claims in the documents. Begin by assessing each claim
    separately, one by one; don’t make any remarks about the response as a whole
    until you have assessed all the claims in isolation.
    The overall_supported field is a boolean indicating whether the response as a
    whole is supported by the documents. This value should reflect the conclusion
    you drew at the end of your step-by-step breakdown in overall_supported_explanation.
    In the sentence_support_information field, provide information about the support
    *for each sentence* in the response.
    The sentence_support_information field is a list of objects, one for each sentence
    in the response. Each object MUST have the following fields:
    - response_sentence_key: a string identifying the sentence in the response.
    This key is the same as the one used in the response above.

    - explanation: a string explaining why the sentence is or is not supported by the
    documents.
    - supporting_sentence_keys: keys (e.g. ’[0a]’) of sentences from the documents that
    support the response sentence. If the sentence is not supported, this list MUST
    be empty. If the sentence is supported, this list MUST contain one or more keys.
    In special cases where the sentence is supported, but not by any specific sentence,
    you can use the string "supported_without_sentence" to indicate that the sentence
    is generally supported by the documents. Consider cases where the sentence is
    expressing inability to answer the question due to lack of relevant information in
    the provided context as "supported_without_sentence". In cases where the sentence
    is making a general statement (e.g. outlining the steps to produce an answer, or
    summarizing previously stated sentences, or a transition sentence), use the
    string "general". In cases where the sentence is correctly stating a well-known fact,
    like a mathematical formula, use the string "well_known_fact". In cases where the
    sentence is performing numerical reasoning (e.g. addition, multiplication), use
    the string "numerical_reasoning".
    - fully_supported: a boolean indicating whether the sentence is fully supported by
    the documents.
    - This value should reflect the conclusion you drew at the end of your step-by-step
    breakdown in explanation.
    - If supporting_sentence_keys is an empty list, then fully_supported must be false.
    - Otherwise, use fully_supported to clarify whether everything in the response
    sentence is fully supported by the document text indicated in supporting_sentence_keys
    (fully_supported = true), or whether the sentence is only partially or incompletely
    supported by that document text (fully_supported = false).
    The all_utilized_sentence_keys field is a list of all sentences keys (e.g. ’0a’) that
    were used to construct the answer. Include every sentence that either directly supported
    the answer, or was implicitly used to construct the answer, even if it was not used
    in its entirety. Omit sentences that were not used, and could have been removed from
    the documents without affecting the answer.
    You must respond with a valid JSON string. Use escapes for quotes, e.g. \\"\\", and
    newlines, e.g. \\n. Do not write anything before or after the JSON string. Do not
    wrap the JSON string in backticks like ‘‘‘ or ‘‘‘json.
    As a reminder: your task is to review the response and assess which documents contain
    useful information pertaining to the question, and how each sentence in the response
    is supported by the text in the documents.
    """.strip()


## **Response generation using groq using llama3-8b-8192**

In [25]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chat = ChatGroq(temperature=0.3, groq_api_key="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", model_name="llama3-70b-8192")

prompt_template_with_docs = PromptTemplate(
    input_variables=["documents", "question", "answer"],
    template=generate_prompt(),
)

print('context for groq >>>> ', context)
print('query for groq >>>> ', query)
print('answer for groq >>>> ', answer)

chain = prompt_template_with_docs | chat

# if classification == 1:
#  groq_response_with_context_qanda = chain.invoke({"documents": context, "question": query, "answer":groq_response})
# else
#  groq_response_with_out-rag = chain.invoke({"documents": "", "question": query, "answer":groq_response})

groq_response_with_context_qanda = chain.invoke({"documents": context, "question": query, "answer":answer})

print("groq_response>>>>with context, query and answer>>>>> ",groq_response_with_context_qanda)

context for groq >>>>  [array('[-1p] Caused by:\n   com.ibm.db2.jcc.am.SqlIntegrityConstraintViolationException:\n   Error for batch element #1: DB2 SQL Error: SQLCODE=-803,\n   SQLSTATE=23505, SQLERRMC=2;DB2ADMIN.LSW_USR_GRP_XREF,\n   DRIVER=4.12.55\n     at com.ibm.db2.jcc.am.hd.a(hd.java:672)\n     at com.ibm.db2.jcc.am.hd.a(hd.java:60)\n     at com.ibm.db2.jcc.am.hd.a(hd.java:127)\n     at com.ibm.db2.jcc.t4.cb.a(cb.java:481)\n     at com.ibm.db2.jcc.t4.cb.a(cb.java:70)\n     at com.ibm.db2.jcc.t4.q.a(q.java:57)\n     at com.ibm.db2.jcc.t4.sb.a(sb.java:225)\n     at com.ibm.db2.jcc.am.nn.a(nn.java:3089)\n     at com.ibm.db2.jcc.am.nn.d(nn.java:5049)\n     at com.ibm.db2.jcc.am.nn.a(nn.java:4494)\n     at com.ibm.db2.jcc.am.nn.c(nn.java:4320)\n     at com.ibm.db2.jcc.am.nn.executeBatch(nn.java:2589)\n     at\n   com.ibm.ws.rsadapter.jdbc.WSJdbcPreparedStatement.pmiExecuteBatc\n   h(WSJdbcPreparedStatement.java:1035)\n     at\n   com.ibm.ws.rsadapter.jdbc.WSJdbcStatement.executeBatch

## **JSON Data parsing to retrieve metrics**

In [ ]:
import re
import json

In [27]:
# Extract the content field using regular expressions
content_match = re.search(r"content='(.*?)' additional_kwargs=", str(groq_response_with_context_qanda), re.DOTALL)
if content_match:
    content = content_match.group(1)
    print("Extracted Content:")
    print(content)

    json_match = re.search(r"\{.*\}", content, re.DOTALL)
    if json_match:
      json_str = json_match.group(0)

      json_str = json_str.replace("'", '"').replace("\\n","").replace("\\","")
      print(json_str)
      try:
        # Parse the JSON
        parsed_json = json.loads(json_str)
        print("Extracted JSON:")
        print(json.dumps(parsed_json, indent=4))

      except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
    else:
      print("No JSON found in the provided string.")

else:
    print("Content field not found in the provided string.")


Extracted Content:
Here is the response:\n\n{\n"relevance_explanation": "Document [-1b] contains useful information for answering the question, as it provides a detailed explanation of the error message and the solution to the problem. Document [-1a] also contains useful information, as it provides the error message and the context in which it occurred.",\n\'all_relevant_sentence_keys\': [\'[-1b]\', \'[0a]\', \'[0b]\', \'[1a]\', \'[1b]\', \'[1g]\', \'[1g]\', \'[1a]\', \'[1b]\', \'[1g]\', \'[1a]\', \'[1b]\', \'[1g]\', \'[1a]\', \'[1b]\', \'[1g]\', \'[1a]\', \'[1b]\', \'[1g]\', \'[1a]\', \'[1b]\', \'[1g]\', \'[1a]\', \'[1b]\', \'[1g]\', \'[1a]\', \'[1b]\', \'[1g]\', \'[1a]\', \'[1b]\', \'[1g]\', \'[1a]\', \'[1b]\', \'[1g]\', \'[1a]\', \'[1b]\', \'[1g]\', \'[1a]\', \'[1b]\', \'[1g]\', \'[1a]\', \'[1b]\', \'[1g]\', \'[1a]\', \'[1b]\', \'[1g]\', \'[1a]\', \'[1b]\', \'[1g]\', \'[1a]\', \'[1b]\', \'[1g]\', \'[1a]\', \'[1b]\', \'[1g]\', \'[1a]\', \'[1b]\', \'[1g]\', \'[1a]\', \'[1b]\', \'[1g]\

In [ ]:
data = parsed_json

## **Computation Metrics from JSON response in comparison with ground truth**

In [ ]:
import json

In [ ]:
# Helper function for length computation (mocked as sentence count here)
def compute_length(keys):
    return len(keys)

# Metrics Computation
def compute_metrics(data):
    all_relevant = data["all_relevant_sentence_keys"]
    all_utilized = data["all_utilized_sentence_keys"]
    sentences_info = data["sentence_support_information"]

    # Context Relevance
    total_relevant_length = compute_length(all_relevant)
    total_context_length = total_relevant_length  # Assuming all relevant are part of the context
    context_relevance = total_relevant_length / total_context_length if total_context_length > 0 else 0

    # Context Utilization
    total_utilized_length = compute_length(all_utilized)
    context_utilization = total_utilized_length / total_context_length if total_context_length > 0 else 0

    # Completeness
    total_relevant_utilized = sum(
        1 for s in sentences_info if set(s["supporting_sentence_keys"]).intersection(all_utilized)
    )
    completeness = total_relevant_utilized / total_relevant_length if total_relevant_length > 0 else 0

    # Adherence
    adherence = all(s["fully_supported"] for s in sentences_info)

    return {
        "Context Relevance": context_relevance,
        "Context Utilization": context_utilization,
        "Completeness": completeness,
        "Adherence": adherence
    }

In [ ]:
# Compute and print metrics
predicted_metrics = compute_metrics(data)
print(json.dumps(predicted_metrics, indent=4))

## **Fetching the groud truth values**

In [ ]:
sub_dataset = load_dataset("rungalileo/ragbench", "covidqa")

In [ ]:
def fetch_ground_truth(dataset, question):
    for sample in dataset["train"]:  # Change "train" to the correct split if needed
        if sample["question"] == question:
            return {
                "Context Relevance": sample["relevance_score"],  # Adjust column name if needed
                "Context Utilization": sample["utilization_score"],  # Adjust column name if needed
                "Adherence": sample["adherence_score"]  # Adjust column name if needed
            }
    return None


In [ ]:
ground_truth = fetch_ground_truth(sub_dataset, query)

if ground_truth:
    print("Ground Truth Values:")
    print(json.dumps(ground_truth, indent=4))
else:
    print(f"Question not found in the dataset: {query}")

In [ ]:
## DONOT RUN THIS
ground_truth = """{
    "Context Relevance": 0.6470588235294118,
    "Context Utilization": 0.35294117647058826,
    "Adherence": false
}"""

In [ ]:
## DONOT RUN THIS
predicted_metrics = """{
    "Context Relevance": 1.0,
    "Context Utilization": 0.6,
    "Completeness": 1.0,
    "Adherence": true
}"""

## **Evaluation Metrics**

In [ ]:
from sklearn.metrics import mean_squared_error, roc_auc_score
import numpy as np
import json

# **RMSE**

In [ ]:
def compute_rmse(predicted, ground_truth):

    # Extract true and predicted values
    y_true_relevance = ground_truth["Context Relevance"]
    y_true_utilization = ground_truth["Context Utilization"]

    y_pred_relevance = predicted["Context Relevance"]
    y_pred_utilization = predicted["Context Utilization"]

    # Compute RMSE for Context Relevance and Context Utilization
    rmse_relevance = np.sqrt((y_pred_relevance - y_true_relevance) ** 2)
    rmse_utilization = np.sqrt((y_pred_utilization - y_true_utilization) ** 2)

    return {
        "RMSE-Relevance": rmse_relevance,
        "RMSE-Utililization": rmse_utilization,
    }


# **AUCROC**

In [ ]:
#The AUC-ROC is a metric for binary classification, but it requires:
# 1) Both positive and negative classes in the ground truth.
# 2) A set of predictions (not just a single value).
# The AUR-ROC code that we have written earlier will always fail because at any given point of time only one class (either true or false) of ground truth and predicted value is
# passed to the roc_auc_score library function. However, by definition, AUC-ROC requires a 'set of 2 class' values. Following is a toy example
# TODO : For our project metric, we need to pass 2 sets of values to roc_auc_score function
  # 1) multiple adherence values of ground truth queries
  # 2) multiple adherence values from our predictions

y_true = ["true","false","false","true"]  # Ground truth with both classes
y_pred = ["true","true","true","false"]  # Model probabilities

mapping = {"true": 1, "false": 0}
y_true_numeric_alt = [mapping[val] for val in y_true]
y_pred_numeric_alt = [mapping[val] for val in y_pred]

auc_roc = roc_auc_score(y_true_numeric_alt, y_pred_numeric_alt)
print("AUC-ROC:", auc_roc)

In [ ]:
evaluation_metrics = compute_rmse(predicted_metrics, ground_truth)

In [ ]:
# Print Results
print("Ground Truth Values (JSON):")
print(json.dumps(ground_truth, indent=4))
print("\nPredicted Metrics:")
print(json.dumps(predicted_metrics, indent=4))
print("\nEvaluation Metrics (RMSE and AUC-ROC):")
print(json.dumps(evaluation_metrics, indent=4))